<a href="https://colab.research.google.com/github/Alirezamirbagheri/Machine-Learning-Transfer-learning-for-configurations-control/blob/main/Transfer_Learning_for_Configuration_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Libraries**

In [1]:
import numpy as np
import PIL.Image as Image
import matplotlib.pylab as plt
import torch
from torch.utils.data import Dataset
import pathlib
from torchsummary import summary
from tqdm import tqdm
from torch.optim import Adam
import torch.nn as nn
from torchvision.transforms import transforms
import os
!pip install torchmetrics
from torchmetrics import ConfusionMatrix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 27.3 MB/s eta 0:00:00


In [2]:
print("Numpy version: " + np.__version__)
print("PIL.Image version: " + Image.__version__)
print("Matplotlib.pylab version: " + plt.__version__)

Numpy version: 2.0.2
PIL.Image version: 11.3.0
Matplotlib.pylab version: 2.0.2


# **2. Data preparation and import**
In this Notebook Google Drive is used.

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 3.1 **Loading the test data**
The data set is already split into a **training**, **validation** and  **test** sets.  The class names are derived from the sub folder names.

For a CNN, the images must have the same dimensions (height × width × channels).
Because:
- Convolution layers slide fixed-size filters over the image.
- If images had different sizes, the output feature maps would also have different shapes → impossible to batch them together.
- Dense (fully connected) layers at the end require a fixed input size.

Here we define a variable called **IMAGE_SHAPE**.

In [7]:
import tensorflow as tf
IMAGE_SHAPE = (224, 224)
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
training_data="./drive/MyDrive/ML/Transfer_Learning/dataset_truck/training"
training_image_data  = image_generator.flow_from_directory(training_data,target_size=IMAGE_SHAPE)

# rescale=1./255 → normalizes pixel values from [0,255] → [0,1].
# target_size=(224,224) → resizes every image to the input size required by your CNN (e.g., ResNet, MobileNet, VGG all expect 224×224).
# flow_from_directory → automatically assigns labels from folder names inside training_data.

Found 60 images belonging to 6 classes.
